# Preliminaries

In [23]:
from fastquant import get_stock_data, get_stock_table
import pandas as pd
import numpy as np

#Forecasting
from pandas.plotting import lag_plot
from pandas import datetime
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

#Optimization
import scipy.optimize as optimization

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


Fastquant allows us to get the info on Philippine stocks compared to other API such as the 'tidyquant' package. One example that comes to mind is that using the ticker for PLDT which is TEL, refers to other global stocks and may not be easily retrievable.

## DATA PROCESSING

### GET THE STOCK DATA

We get the stock data, to track the volatility of the market. Ideally, we want to a time-period that covers the full nusiness cycle, from trough, recession, expansion and peak.

To begin, specify the stock and past trading dates you are looking at. The past trading dates will provide us the riskiness of the stocks through the standard deviation of its returns. 

### Specify the stocks you are following

In [100]:
stocks = ['MEG','CEB', 'BDO','ALI', 'MER', 'AC', 'JGS', 'URC', 
          'JFC', 'SEVN', 'BMM', 'CAT', 'FGEN', 'MAXS', 'PGOLD']

In [101]:
START_DATE = "2014-01-01" #yyyy-mm-dd
END_DATE = "2019-12-29"

df = pd.DataFrame()
for stock in stocks:
    try:
        df_ = get_stock_data(stock, START_DATE, END_DATE)
    except:
        continue
    df_.columns = [stock]
    df = pd.concat([df, df_], axis=1)

In [102]:
df

,MEG,CEB,BDO,ALI,MER,AC,JGS,URC,JFC,SEVN,BMM,CAT,FGEN,MAXS,PGOLD
dt,,,,,,,,,,,,,,,
2014-01-02,3.31,49.00,71.00,25.50,256.0,525.5,39.00,118.0,175.9,NaN,NaN,NaN,14.02,15.10,39.00
2014-01-03,3.31,49.40,70.45,25.25,254.2,520.5,38.85,119.0,172.0,98.0,NaN,NaN,14.14,15.30,39.20
2014-01-06,3.41,49.50,71.80,25.50,260.0,525.5,39.50,119.4,175.0,97.0,NaN,NaN,14.28,15.20,39.00
2014-01-07,3.40,49.25,71.85,25.60,257.0,524.0,38.20,115.5,172.0,95.0,NaN,NaN,14.36,15.10,39.10
2014-01-08,3.38,50.95,72.55,26.10,263.0,530.5,39.20,118.7,169.5,95.0,NaN,NaN,14.86,14.64,39.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-19,4.10,90.70,154.80,45.40,311.4,755.0,77.00,143.0,216.0,148.0,NaN,17.54,23.50,12.10,40.40
2019-12-20,4.00,90.40,155.40,46.00,286.2,779.5,80.10,144.0,213.0,NaN,NaN,17.52,22.90,12.10,39.95
2019-12-23,4.03,88.90,157.00,47.25,299.8,779.0,81.70,151.9,220.0,145.5,NaN,17.52,23.75,12.12,39.95


In [103]:
#Save the data
# df.to_csv("data/stocks_closing_prices.csv")

In [81]:
# df = pd.read_csv("data/stocks_closing_prices.csv",index_col='dt')
# df.head()

,MEG,CEB,BDO,ALI,MER,AC,JGS,URC,JFC,SEVN,BMM,CAT,FGEN
dt,,,,,,,,,,,,,
2015-01-05,4.81,88.20,110.0,34.00,256.6,697.0,69.00,191.0,211.0,NaN,NaN,93.85,26.00
2015-01-06,5.00,86.55,108.5,33.50,262.8,701.0,69.00,191.5,216.0,83.95,NaN,NaN,26.20
2015-01-07,5.09,84.00,106.6,33.75,264.0,701.5,69.50,196.9,216.0,83.00,NaN,NaN,26.50
2015-01-08,5.18,79.10,105.8,35.25,265.0,710.0,70.00,196.0,215.8,NaN,NaN,NaN,26.50
2015-01-09,5.18,84.00,106.0,35.00,262.0,715.0,69.95,196.5,209.6,NaN,44.6,93.65,26.35


Portfolio maximization is a quadratic optimization problem. Because we want to optimize the trade-off between returns and variance, we need to have variables that relate to these two. 

For this, we would need the following:
  *  Returns data
  *  Covariance matrix for the **returns**
  *  weights allocated for each of the stock

Therefore, we need to get the returns of the stock prices that we have. Since the asset prices are stored in a pandas dataframe, then we can simply call on the method pct_change().

### Returns Calculation

In [104]:
returns = df.pct_change()
returns.head()

,MEG,CEB,BDO,ALI,MER,AC,JGS,URC,JFC,SEVN,BMM,CAT,FGEN,MAXS,PGOLD
dt,,,,,,,,,,,,,,,
2014-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-03,0.000000,0.008163,-0.007746,-0.009804,-0.007031,-0.009515,-0.003846,0.008475,-0.022172,NaN,NaN,NaN,0.008559,0.013245,0.005128
2014-01-06,0.030211,0.002024,0.019163,0.009901,0.022817,0.009606,0.016731,0.003361,0.017442,-0.010204,NaN,NaN,0.009901,-0.006536,-0.005102
2014-01-07,-0.002933,-0.005051,0.000696,0.003922,-0.011538,-0.002854,-0.032911,-0.032663,-0.017143,-0.020619,NaN,NaN,0.005602,-0.006579,0.002564
2014-01-08,-0.005882,0.034518,0.009743,0.019531,0.023346,0.012405,0.026178,0.027706,-0.014535,0.000000,NaN,NaN,0.034819,-0.030464,0.002558


### Covariance Matrix

In [105]:
cov = returns.cov()
cov.head()

,MEG,CEB,BDO,ALI,MER,AC,JGS,URC,JFC,SEVN,BMM,CAT,FGEN,MAXS,PGOLD
MEG,0.000435,0.000087,0.000105,0.000141,0.000059,0.000119,0.000138,0.000092,0.000081,-1.695857e-05,0.000010,0.000031,0.000077,0.000080,0.000089
CEB,0.000087,0.000554,0.000063,0.000060,0.000024,0.000049,0.000068,0.000056,0.000031,-3.655724e-07,0.000017,0.000023,0.000062,0.000063,0.000044
BDO,0.000105,0.000063,0.000231,0.000095,0.000051,0.000076,0.000106,0.000092,0.000062,-2.180059e-06,0.000018,0.000003,0.000045,0.000040,0.000054
ALI,0.000141,0.000060,0.000095,0.000303,0.000052,0.000124,0.000130,0.000108,0.000087,8.034783e-06,0.000028,0.000005,0.000070,0.000068,0.000061
MER,0.000059,0.000024,0.000051,0.000052,0.000194,0.000045,0.000071,0.000055,0.000043,4.941043e-07,0.000030,0.000020,0.000033,0.000011,0.000029


### Portfolio Weights

It is important that the weights chosen come from a uniform distribution from 0 to 1. Luckily, random.random of numpy draws from a continuous, uniform distribution.

In [140]:
weights = np.random.random(len(stocks))
weights /= np.sum(weights)
weights

array([0.0839457 , 0.12381237, 0.06088195, 0.11016387, 0.09311314,
       0.00604991, 0.08074676, 0.02290855, 0.11069774, 0.02103943,
       0.12906059, 0.10749034, 0.02550521, 0.0072724 , 0.01731204])

In [141]:
#Portfolio Return
rp = (returns.mean()*252)@weights 
rp

0.22122431920685026

In [142]:
#Portfolio Variance
port_var = weights@(cov*252)@weights 
port_var

0.04378754990769984

In [143]:
#Sharpe Ratio
rf = 0.02 #risk-free rate
sharpe = (rp-rf)/np.sqrt(port_var)
sharpe

0.961623668434884

## OPTIMIZATION PROBLEM

In an optimization problem, we need the following:
    
  *  objective function - this is the function which we aim to minimize
  *  decision variables - these are the variables which we change in order to achieve the optimal output. In our case, we need to change the weights to produce an optimal output.
  *  constraints - a list of decision constraints

In [161]:
#Objective Function
def min_sharpe_ratio(weights):
    return -portfolio_metrics(weights).loc['MV-Optimal Portfolio','Sharpe Ratio']

#Constraints
constraints = [{'type':'eq','fun': lambda x: np.sum(x)-1}]

#Bounds
bounds = [(0, 1) for i in range(len(stocks))] 



In [186]:
opt_weights = optimization.minimize(fun=min_sharpe_ratio,x0=weights,method='SLSQP',bounds=bounds,constraints=constraints)['x']

In [187]:
def portfolio_metrics(weights):
    
    '''
    This function generates the relative performance metrics that will be reported and will be used
    to find the optimal weights.
    
    Parameters:
    weights: initialized weights or optimal weights for performance reporting
    
    '''   
    
    rp = (returns.mean()*252)@weights 
    port_var = weights@(cov*252)@weights
    sharpe = (rp-rf)/np.sqrt(port_var)
    df = pd.DataFrame({"Expected Return": rp,
                       "Portfolio Variance":port_var,
                       'Portfolio Std': np.sqrt(port_var),
                       'Sharpe Ratio': sharpe}, index=["MV-Optimal Portfolio"])
    return df

portfolio_metrics(opt_weights)

,Expected Return,Portfolio Variance,Portfolio Std,Sharpe Ratio
MV-Optimal Portfolio,0.231148,0.034845,0.186669,1.131134


In [191]:
def optimal_portfolio_weights(weights):
    df = pd.DataFrame({"Stock": stocks,
                       "Weights": weights})
    df.Weights = df.Weights.map(lambda x: '{:.2%}'.format(x))
    return df

optimal_portfolio_weights(opt_weights)

,Stock,Weights
0,MEG,0.00%
1,CEB,10.63%
2,BDO,27.79%
3,ALI,4.74%
4,MER,0.00%
5,AC,0.00%
6,JGS,12.46%
7,URC,0.00%
8,JFC,0.00%
9,SEVN,12.10%


References:
    
    https://enzoampil.github.io/fastquant-blog/portfolio/optimization/2020/06/20/basic_portfolio.html
        
    https://www.codingfinance.com/post/2018-05-31-portfolio-opt-in-r/
    
    https://towardsdatascience.com/time-series-forecasting-predicting-stock-prices-using-an-arima-model-2e3b3080bd70